## XGBoost uses decision trees to perform the predictions

In [3]:
# Import the necessary modules
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X,y=data.data, data.target
import numpy as np

# Create the training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=123)

# Instantiate the classifier: dt_clf_4
dt_clf_4 = DecisionTreeClassifier(max_depth=4)
                     #max_depth:pecifies the maximum number of successive split points you can have before reaching a leaf node

# Fit the classifier to the training set
dt_clf_4.fit(X_train,y_train)

# Predict the labels of the test set: y_pred_4
y_pred_4 = dt_clf_4.predict(X_test)

# Compute the accuracy of the predictions: accuracy
accuracy = float(np.sum(y_pred_4==y_test))/y_test.shape[0]
print("accuracy:", accuracy)

accuracy: 0.9736842105263158


## Using XGBoost and Cross validation with errror metric

In [13]:
import xgboost as xgb

# Create the DMatrix from X and y: churn_dmatrix
churn_dmatrix = xgb.DMatrix(data=X, label=y)

# Create the parameter dictionary: params
params = {"objective":"binary:logistic", "max_depth":3}

# Perform cross-validation: cv_results
cv_results = xgb.cv(dtrain=churn_dmatrix, params=params, 
                  nfold=100, num_boost_round=100, 
                  metrics="error", as_pandas=True, seed=123)

# Print cv_results
print(cv_results)

# Print the accuracy
print(((1-cv_results["test-error-mean"]).iloc[-1]))

    train-error-mean  train-error-std  test-error-mean  test-error-std
0           0.026167         0.003495         0.093333        0.123288
1           0.017255         0.003301         0.064667        0.097162
2           0.015675         0.001979         0.062667        0.096411
3           0.012923         0.002258         0.056333        0.092219
4           0.011450         0.002089         0.052333        0.087083
..               ...              ...              ...             ...
95          0.000000         0.000000         0.029333        0.073224
96          0.000000         0.000000         0.029333        0.073224
97          0.000000         0.000000         0.029333        0.073224
98          0.000000         0.000000         0.029333        0.073224
99          0.000000         0.000000         0.029333        0.073224

[100 rows x 4 columns]
0.97066664


In [15]:
# Perform cross_validation: cv_results
cv_results = xgb.cv(dtrain=churn_dmatrix, params=params, 
                  nfold=100, num_boost_round=100, 
                  metrics="auc", as_pandas=True, seed=123)

# Print cv_results
print(cv_results)

# Print the AUC
print((cv_results["test-auc-mean"]).iloc[-1])




#hay un error con la metrica AUC

[10:35:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/metric/auc.cc:307: Dataset contains only positive or negative samples.
[10:35:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/metric/auc.cc:307: Dataset contains only positive or negative samples.
[10:35:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/metric/auc.cc:307: Dataset contains only positive or negative samples.
[10:35:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/metric/auc.cc:307: Dataset contains only positive or negative samples.
[10:35:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/metric/auc.cc:307: Dataset contains only positive or negative samples.
[10:35:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/metric/auc.cc:307: Dataset contains only positive or negative samples.
[10:35:32] WARNING: C:/Users/Administrator/workspace/xgboost-win